# 📚 Workshop: RAG + LangChain + Streamlit

## ช่วงที่ 3: Build RAG Pipeline with LangChain (1:30 – 2:30)

---

### 🎯 วัตถุประสงค์การเรียนรู้
- ใช้ RetrievalQA หรือ ConversationalRetrievalChain
- ตั้งค่า retriever (search_type="similarity", k=3)
- เชื่อมต่อกับ LLM (Groq Llama)
- ทดลองถาม-ตอบจากข้อมูล
- สร้าง Custom Prompt Template

---


## 🔗 RAG Pipeline Overview

### สถาปัตยกรรม RAG Pipeline:

```mermaid
graph TD
    A[คำถามของผู้ใช้] --> B[Embeddings]
    B --> C[Vector Search]
    D[Vector Store] --> C
    C --> E[Retriever]
    E --> F[Context Documents]
    F --> G[Prompt Template]
    G --> H[LLM]
    H --> I[คำตอบ]
```

### LangChain Chains:

1. **RetrievalQA** - สำหรับคำถาม-คำตอบแบบง่าย
2. **ConversationalRetrievalChain** - สำหรับการสนทนาที่มี memory
3. **RetrievalQAWithSourcesChain** - รวมแหล่งอ้างอิง


In [1]:
# Import Libraries และโหลด Vector Store
import os
import sys
from pathlib import Path

# เพิ่ม path สำหรับ import modules
sys.path.append('..')

# Import LangChain components
from langchain.chains import RetrievalQA
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Import utilities
from dotenv import load_dotenv
import logging

# ตั้งค่า logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# โหลด environment variables
load_dotenv()

print("✅ Libraries imported successfully!")

# โหลด Vector Store (ถ้ามี)
vectorstore_path = "../vectorstore"
if os.path.exists(vectorstore_path):
    try:
        # โหลด embeddings
        embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            cache_folder="../model_cache"
        )
        
        # โหลด FAISS vector store
        vectorstore = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
        print(f"✅ โหลด Vector Store สำเร็จ: {vectorstore.index.ntotal} documents")
    except Exception as e:
        print(f"❌ ไม่สามารถโหลด Vector Store: {e}")
        vectorstore = None
else:
    print("❌ ไม่พบ Vector Store - กรุณารัน notebook ช่วงที่ 2 ก่อน")
    vectorstore = None


/home/game/01_Projects/workshop_rag_rmutl/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


/tmp/ipykernel_40136/1761979277.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


✅ โหลด Vector Store สำเร็จ: 13 documents


## 🤖 ตั้งค่า LLM (Groq)

### Groq คืออะไร?

**Groq** เป็น AI inference company ที่ให้บริการ LLM inference ที่เร็วมาก

### ข้อดีของ Groq:
- ⚡ **เร็วมาก** - inference เร็วกว่า OpenAI
- 💰 **ราคาถูก** - ราคาต่อ token ต่ำกว่า
- 🔓 **Open Source** - รองรับโมเดล open source
- 🌐 **API** - ใช้งานง่ายผ่าน API

### โมเดลที่แนะนำ:
- `llama-3.3-70b-versatile` - โมเดลใหญ่, คุณภาพดี
- `llama-3.1-8b-instant` - โมเดลเล็ก, เร็ว
- `mixtral-8x7b-32768` - Mixtral model


In [2]:
# ตั้งค่า LLM (Groq)
def setup_llm(model_name="llama-3.3-70b-versatile", temperature=0.1):
    """ตั้งค่า Groq LLM"""
    try:
        # ตรวจสอบ API Key
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            raise ValueError("GROQ_API_KEY ไม่พบใน environment variables")
        
        # สร้าง Groq LLM
        llm = ChatGroq(
            groq_api_key=api_key,
            model_name=model_name,
            temperature=temperature
        )
        
        print(f"✅ ตั้งค่า Groq LLM สำเร็จ")
        print(f"🤖 Model: {model_name}")
        print(f"🌡️ Temperature: {temperature}")
        
        # ทดสอบ LLM
        test_response = llm.invoke("สวัสดี")
        print(f"🧪 ทดสอบ LLM: {test_response.content[:50]}...")
        
        return llm
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return None

# ตั้งค่า LLM
llm = setup_llm("llama-3.3-70b-versatile", temperature=0.1)


✅ ตั้งค่า Groq LLM สำเร็จ
🤖 Model: llama-3.3-70b-versatile
🌡️ Temperature: 0.1


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🧪 ทดสอบ LLM: สวัสดีครับ/ค่ะ ยินดีที่ได้รับใช้ครับ/ค่ะ มีอะไรที่...


## 🎯 สร้าง Custom Prompt Template

### ทำไมต้องใช้ Custom Prompt?

1. **ควบคุมการตอบ** - ให้ LLM ตอบตามรูปแบบที่ต้องการ
2. **เพิ่มบริบท** - อธิบายบทบาทของ LLM
3. **ปรับปรุงคุณภาพ** - ให้คำตอบที่แม่นยำขึ้น
4. **รองรับภาษาไทย** - ปรับ prompt ให้เหมาะกับภาษาไทย

### โครงสร้าง Prompt Template:

```
System Message: บทบาทของ LLM
Context: ข้อมูลจากเอกสาร
Question: คำถามของผู้ใช้
Answer: คำตอบที่ต้องการ
```


In [ ]:
# สร้าง Custom Prompt Template
def create_custom_prompt():
    """สร้าง Custom Prompt Template สำหรับภาษาไทย"""
    
    prompt_template = """
        คุณเป็นผู้ช่วยที่เชี่ยวชาญเกี่ยวกับข้อมูลในเอกสาร จงตอบคำถามอย่างกระชับและถูกต้อง

        ข้อมูลอ้างอิง:
        {context}

        คำถาม: {question}

        คำตอบ:"""
    
    # สร้าง PromptTemplate
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    print("✅ สร้าง Custom Prompt Template สำเร็จ")
    print("📝 Prompt Template:")
    print("-" * 50)
    print(prompt_template)
    print("-" * 50)
    
    return PROMPT

# สร้าง Prompt Template
custom_prompt = create_custom_prompt()


✅ สร้าง Custom Prompt Template สำเร็จ
📝 Prompt Template:
--------------------------------------------------

คุณเป็นผู้ช่วยที่เชี่ยวชาญเกี่ยวกับข้อมูลในเอกสาร จงตอบคำถามอย่างกระชับและถูกต้อง

ข้อมูลอ้างอิง:
{context}

คำถาม: {question}

คำตอบ:
--------------------------------------------------


## 🔗 สร้าง RetrievalQA Chain

### RetrievalQA Chain คืออะไร?

**RetrievalQA** เป็น chain ที่รวม:
1. **Retriever** - ค้นหาเอกสารที่เกี่ยวข้อง
2. **QA Chain** - สร้างคำตอบจากเอกสารที่ค้นหาได้

### พารามิเตอร์สำคัญ:
- `llm`: Language Model
- `retriever`: Vector Store Retriever
- `chain_type`: ประเภทของ chain ("stuff", "map_reduce", "refine")
- `prompt`: Custom Prompt Template


In [4]:
# สร้าง RetrievalQA Chain
def create_retrieval_qa_chain(vectorstore, llm, custom_prompt, k=3):
    """สร้าง RetrievalQA Chain"""
    try:
        if not vectorstore or not llm:
            raise ValueError("ต้องมี vectorstore และ llm")
        
        # สร้าง retriever
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k}
        )
        
        print(f"🔍 สร้าง Retriever สำเร็จ (k={k})")
        
        # สร้าง RetrievalQA chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": custom_prompt},
            return_source_documents=True
        )
        
        print("✅ สร้าง RetrievalQA Chain สำเร็จ")
        print(f"🔗 Chain Type: stuff")
        print(f"📊 Return Source Documents: True")
        
        return qa_chain
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return None

# สร้าง RetrievalQA Chain
if vectorstore and llm and custom_prompt:
    qa_chain = create_retrieval_qa_chain(vectorstore, llm, custom_prompt, k=3)
else:
    print("❌ ไม่สามารถสร้าง RetrievalQA Chain - ตรวจสอบ vectorstore, llm, และ prompt")
    qa_chain = None


🔍 สร้าง Retriever สำเร็จ (k=3)
✅ สร้าง RetrievalQA Chain สำเร็จ
🔗 Chain Type: stuff
📊 Return Source Documents: True


## 🧪 ทดสอบ RAG Pipeline

### คำถามทดสอบ:
1. "จังหวัดน่านมีประชากรเท่าไหร่?"
2. "จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?"
3. "จังหวัดน่านมีประวัติศาสตร์อย่างไร?"
4. "จังหวัดน่านมีอาหารพื้นเมืองอะไรบ้าง?"


In [5]:
# ทดสอบ RAG Pipeline
def test_rag_pipeline(qa_chain, questions):
    """ทดสอบ RAG Pipeline"""
    if not qa_chain:
        print("❌ ไม่มี QA Chain ให้ทดสอบ")
        return
    
    print("🧪 ทดสอบ RAG Pipeline")
    print("=" * 60)
    
    for i, question in enumerate(questions, 1):
        print(f"\n📝 คำถามที่ {i}: {question}")
        print("-" * 50)
        
        try:
            # รัน QA Chain
            result = qa_chain({"query": question})
            
            # แสดงคำตอบ
            answer = result["result"]
            print(f"🤖 คำตอบ: {answer}")
            
            # แสดงแหล่งอ้างอิง
            source_docs = result.get("source_documents", [])
            if source_docs:
                print(f"\n📚 แหล่งอ้างอิง ({len(source_docs)} เอกสาร):")
                for j, doc in enumerate(source_docs, 1):
                    print(f"  {j}. หน้า {doc.metadata.get('page', 'N/A')}")
                    print(f"     เนื้อหา: {doc.page_content[:100]}...")
            
            print("\n" + "=" * 60)
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            print("\n" + "=" * 60)

# คำถามทดสอบ
test_questions = [
    "จังหวัดน่านมีประชากรเท่าไหร่?",
    "จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?",
    "จังหวัดน่านมีประวัติศาสตร์อย่างไร?",
    "จังหวัดน่านมีอาหารพื้นเมืองอะไรบ้าง?"
]

# ทดสอบ RAG Pipeline
test_rag_pipeline(qa_chain, test_questions)


/tmp/ipykernel_40136/4269863517.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


🧪 ทดสอบ RAG Pipeline

📝 คำถามที่ 1: จังหวัดน่านมีประชากรเท่าไหร่?
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: ข้อมูลที่ให้มาไม่ได้ระบุจำนวนประชากรของจังหวัดน่าน

📚 แหล่งอ้างอิง (3 เอกสาร):
  1. หน้า 2
     เนื้อหา: สมัยกรุงรัตนโกสินทร์ นครน่านมีฐานะเป็นหัวเมืองประเทศราช เจ้าผู้ครองนครน่านในชั้นหลัง
ทุกองค์ต่างปฏิบ...
  2. หน้า 0
     เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาค...
  3. หน้า 2
     เนื้อหา: สมัยล้านนา 
ในปี พ.ศ. 1993 พระเจ้าติโลกราชกษัตริย์นครเชียงใหม่ มีความประสงค์จะครอบครอง
เมืองน่านและแ...


📝 คำถามที่ 2: จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: จังหวัดน่านมีสถานที่ท่องเที่ยวหลายแห่ง เช่น วัดพระธาตุแช่แห้ง, วัดสวนตาล, วัดพระธาตุช้างค้ำ และบ่อเกลือใต้ เป็นต้น

📚 แหล่งอ้างอิง (3 เอกสาร):
  1. หน้า 2
     เนื้อหา: สมัยกรุงรัตนโกสินทร์ นครน่านมีฐานะเป็นหัวเมืองประเทศราช เจ้าผู้ครองนครน่านในชั้นหลัง
ทุกองค์ต่างปฏิบ...
  2. หน้า 0
     เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาค...
  3. หน้า 2
     เนื้อหา: สมัยล้านนา 
ในปี พ.ศ. 1993 พระเจ้าติโลกราชกษัตริย์นครเชียงใหม่ มีความประสงค์จะครอบครอง
เมืองน่านและแ...


📝 คำถามที่ 3: จังหวัดน่านมีประวัติศาสตร์อย่างไร?
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: จังหวัดน่านมีประวัติศาสตร์ที่ยาวนานและซับซ้อน โดยเคยเป็นหัวเมืองประเทศราชในสมัยกรุงรัตนโกสินทร์ และเคยถูกผนวกเข้าไว้ในอาณาจักรล้านนาในสมัยพระเจ้าติโลกราช ก่อนจะถูกพม่าเข้ายึดครองหลายครั้งในช่วงปี พ.ศ. 2103–2328 นอกจากนี้ยังมีการซึมซับเอาศิลปวัฒนธรรมของล้านนาเข้ามาในวิถีชีวิต โดยเฉพาะการรับเอาศิลปกรรมทางด้านศาสนาแบบล้านนาเข้ามาแทนที่ศิลปกรรมแบบสุโขทัย

📚 แหล่งอ้างอิง (3 เอกสาร):
  1. หน้า 2
     เนื้อหา: สมัยกรุงรัตนโกสินทร์ นครน่านมีฐานะเป็นหัวเมืองประเทศราช เจ้าผู้ครองนครน่านในชั้นหลัง
ทุกองค์ต่างปฏิบ...
  2. หน้า 0
     เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาค...
  3. หน้า 2
     เนื้อหา: สมัยล้านนา 
ในปี พ.ศ. 1993 พระเจ้าติโลกราชกษัตริย์นครเชียงใหม่ มีความประสงค์จะครอบครอง
เมืองน่านและแ...


📝 คำถามที่ 4: จังหวัดน่านมีอาหารพื้นเมืองอะไรบ้าง?
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: ข้อมูลที่ให้มาไม่ได้กล่าวถึงอาหารพื้นเมืองของจังหวัดน่าน

📚 แหล่งอ้างอิง (3 เอกสาร):
  1. หน้า 2
     เนื้อหา: สมัยกรุงรัตนโกสินทร์ นครน่านมีฐานะเป็นหัวเมืองประเทศราช เจ้าผู้ครองนครน่านในชั้นหลัง
ทุกองค์ต่างปฏิบ...
  2. หน้า 0
     เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาค...
  3. หน้า 2
     เนื้อหา: สมัยล้านนา 
ในปี พ.ศ. 1993 พระเจ้าติโลกราชกษัตริย์นครเชียงใหม่ มีความประสงค์จะครอบครอง
เมืองน่านและแ...



## 💬 สร้าง ConversationalRetrievalChain

### ConversationalRetrievalChain คืออะไร?

**ConversationalRetrievalChain** เป็น chain ที่มี memory สำหรับการสนทนา:
1. **Memory** - จดจำการสนทนาก่อนหน้า
2. **Retriever** - ค้นหาเอกสารที่เกี่ยวข้อง
3. **LLM** - สร้างคำตอบโดยพิจารณาประวัติการสนทนา

### ข้อดี:
- ✅ จดจำบริบทการสนทนา
- ✅ ตอบคำถามต่อเนื่องได้
- ✅ มีความต่อเนื่องในการสนทนา


In [6]:
# สร้าง ConversationalRetrievalChain
def create_conversational_chain(vectorstore, llm, k=3):
    """สร้าง ConversationalRetrievalChain"""
    try:
        if not vectorstore or not llm:
            raise ValueError("ต้องมี vectorstore และ llm")
        
        # สร้าง memory
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        print("💾 สร้าง Memory สำเร็จ")
        
        # สร้าง retriever
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k}
        )
        
        print(f"🔍 สร้าง Retriever สำเร็จ (k={k})")
        
        # สร้าง ConversationalRetrievalChain
        conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            return_source_documents=True
        )
        
        print("✅ สร้าง ConversationalRetrievalChain สำเร็จ")
        
        return conversation_chain
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return None

# สร้าง ConversationalRetrievalChain
if vectorstore and llm:
    conversation_chain = create_conversational_chain(vectorstore, llm, k=3)
else:
    print("❌ ไม่สามารถสร้าง ConversationalRetrievalChain - ตรวจสอบ vectorstore และ llm")
    conversation_chain = None


💾 สร้าง Memory สำเร็จ
🔍 สร้าง Retriever สำเร็จ (k=3)
✅ สร้าง ConversationalRetrievalChain สำเร็จ


/tmp/ipykernel_40136/2467826205.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


## 🗣️ ทดสอบการสนทนา

### ทดสอบการสนทนาต่อเนื่อง


In [7]:
# ทดสอบการสนทนาต่อเนื่อง
def test_conversation(conversation_chain, conversation_flow):
    """ทดสอบการสนทนาต่อเนื่อง"""
    if not conversation_chain:
        print("❌ ไม่มี Conversation Chain ให้ทดสอบ")
        return
    
    print("🗣️ ทดสอบการสนทนาต่อเนื่อง")
    print("=" * 60)
    
    for i, question in enumerate(conversation_flow, 1):
        print(f"\n👤 ผู้ใช้: {question}")
        print("-" * 30)
        
        try:
            # รัน conversation chain
            result = conversation_chain({"question": question})
            
            # แสดงคำตอบ
            answer = result["answer"]
            print(f"🤖 Bot: {answer}")
            
            # แสดงแหล่งอ้างอิง
            source_docs = result.get("source_documents", [])
            if source_docs:
                print(f"\n📚 แหล่งอ้างอิง ({len(source_docs)} เอกสาร):")
                for j, doc in enumerate(source_docs, 1):
                    print(f"  {j}. หน้า {doc.metadata.get('page', 'N/A')}")
                    print(f"     เนื้อหา: {doc.page_content[:80]}...")
            
            print("\n" + "=" * 60)
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            print("\n" + "=" * 60)

# การสนทนาต่อเนื่อง
conversation_flow = [
    "จังหวัดน่านมีประชากรเท่าไหร่?",
    "แล้วมีพื้นที่เท่าไหร่?",
    "มีเขตการปกครองอะไรบ้าง?",
    "สถานที่ท่องเที่ยวที่สำคัญคืออะไร?"
]

# ทดสอบการสนทนา
test_conversation(conversation_chain, conversation_flow)


🗣️ ทดสอบการสนทนาต่อเนื่อง

👤 ผู้ใช้: จังหวัดน่านมีประชากรเท่าไหร่?
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


❌ เกิดข้อผิดพลาด: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.


👤 ผู้ใช้: แล้วมีพื้นที่เท่าไหร่?
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


❌ เกิดข้อผิดพลาด: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.


👤 ผู้ใช้: มีเขตการปกครองอะไรบ้าง?
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


❌ เกิดข้อผิดพลาด: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.


👤 ผู้ใช้: สถานที่ท่องเที่ยวที่สำคัญคืออะไร?
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


❌ เกิดข้อผิดพลาด: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.



## 🔧 ปรับแต่ง RAG Pipeline

### การปรับแต่งพารามิเตอร์:

1. **k (จำนวนเอกสารที่ค้นหา)**
   - `k=1`: เอกสารเดียว, เร็ว แต่ข้อมูลน้อย
   - `k=3`: สมดุลระหว่างความเร็วและข้อมูล
   - `k=5`: ข้อมูลมาก แต่ช้า

2. **Temperature**
   - `0.0`: ตอบแบบ deterministic
   - `0.1`: ตอบแบบสม่ำเสมอ
   - `0.7`: ตอบแบบสร้างสรรค์

3. **Chain Type**
   - `stuff`: รวมเอกสารทั้งหมดใน prompt เดียว
   - `map_reduce`: แบ่งเอกสารเป็นส่วนๆ แล้วรวมผลลัพธ์
   - `refine`: ปรับปรุงคำตอบทีละเอกสาร


In [8]:
# ทดสอบการปรับแต่งพารามิเตอร์
def test_different_k_values(vectorstore, llm, custom_prompt, question, k_values=[1, 3, 5]):
    """ทดสอบค่า k ต่างๆ"""
    print(f"🔧 ทดสอบค่า k ต่างๆ สำหรับคำถาม: {question}")
    print("=" * 70)
    
    for k in k_values:
        print(f"\n📊 k = {k}")
        print("-" * 30)
        
        try:
            # สร้าง retriever สำหรับ k นี้
            retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": k}
            )
            
            # สร้าง QA chain
            qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=retriever,
                chain_type_kwargs={"prompt": custom_prompt},
                return_source_documents=True
            )
            
            # ทดสอบ
            result = qa_chain({"query": question})
            answer = result["result"]
            source_docs = result.get("source_documents", [])
            
            print(f"🤖 คำตอบ: {answer[:200]}...")
            print(f"📚 จำนวนเอกสารอ้างอิง: {len(source_docs)}")
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
        
        print("\n" + "=" * 70)

# ทดสอบค่า k ต่างๆ
if vectorstore and llm and custom_prompt:
    test_question = "จังหวัดน่านมีประชากรเท่าไหร่?"
    test_different_k_values(vectorstore, llm, custom_prompt, test_question, [1, 3, 5])
else:
    print("❌ ไม่สามารถทดสอบได้ - ตรวจสอบ vectorstore, llm, และ prompt")


🔧 ทดสอบค่า k ต่างๆ สำหรับคำถาม: จังหวัดน่านมีประชากรเท่าไหร่?

📊 k = 1
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: ไม่มีข้อมูลเกี่ยวกับจำนวนประชากรของจังหวัดน่านในข้อมูลที่ให้มา...
📚 จำนวนเอกสารอ้างอิง: 1


📊 k = 3
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: ข้อมูลที่ให้มาไม่ได้ระบุจำนวนประชากรของจังหวัดน่าน...
📚 จำนวนเอกสารอ้างอิง: 3


📊 k = 5
------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


🤖 คำตอบ: ไม่มีข้อมูลเกี่ยวกับจำนวนประชากรของจังหวัดน่านในข้อมูลที่ให้มา...
📚 จำนวนเอกสารอ้างอิง: 5



## 📝 สรุปช่วงที่ 3

### สิ่งที่เราได้เรียนรู้:

1. **RAG Pipeline** 🔗
   - สร้าง RetrievalQA Chain
   - สร้าง ConversationalRetrievalChain
   - ใช้ Custom Prompt Template

2. **LLM Integration** 🤖
   - ตั้งค่า Groq LLM
   - เชื่อมต่อกับ Vector Store
   - ทดสอบการตอบคำถาม

3. **Memory & Conversation** 💬
   - ใช้ ConversationBufferMemory
   - ทดสอบการสนทนาต่อเนื่อง
   - จดจำบริบทการสนทนา

4. **Parameter Tuning** 🔧
   - ปรับแต่งค่า k (จำนวนเอกสาร)
   - ทดสอบพารามิเตอร์ต่างๆ
   - หาค่าที่เหมาะสม

### Pipeline ที่สมบูรณ์:
```
Question → Embeddings → Vector Search → Retriever → Prompt → LLM → Answer
```

### ต่อไปในช่วงที่ 4:
เราจะสร้าง Streamlit UI สำหรับ RAG Chatbot

---

**⏰ เวลาที่ใช้: 60 นาที**
**📚 ไฟล์ต่อไป: `04_streamlit_ui.ipynb`**
